# Batch Data Ingestion for Near real-time sales forecasting leveraging Synapse Link for Azure Cosmos DB

## Key Information about this notebook

* This notebook is part of the Azure Synapse Link for Azure Cosmos DB analitycal sample notebooks. For more information, click [here](../../../README.md). 

* It was build for Azure Cosmos DB SQL API but you can, by yourself, customize it for Azure Cosmos DB API for MongoDB. Please read about the analytical store inference schema differences between these 2 APIs [here](https://docs.microsoft.com/azure/cosmos-db/analytical-store-introduction#analytical-schema). 

* This is a Synapse Notebook and it was created to run in Synapse Analytics workspaces. Please make sure that you followed the pre-reqs of the [README](/README.md) file. After that, please execute the steps below in the same order that they are presented here. 

* From now on, all operations are case sentitive. Please be careful with everything you need to type.

## Predictive Analytics

Predictive analytics can help us to study and discover the factors that determine the number of sales that a retail store will have in the future. This notebook scenario is [Microsoft Surface](https://www.microsoft.com/en-us/surface) sales forecasting, with artificially created data. The business challenge is a **distributor that wants to predict how many units are necessary in the local warehouse to supply the stores in the area.**

We will use Quantitative Models to forecast future data as a function of past data. They are appropriate to use when past numerical data is available and when it is reasonable to assume that some of the patterns in the data are expected to continue into the future. These methods are usually applied to short or intermediate range decisions. For more information, click [here](https://en.wikipedia.org/wiki/Forecasting).


<img src="https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/store.PNG" alt="Surface Device" width="75%"/>

## Environment Creation

### 1. Using the **Data / Linked** tab of your Synapse workspace, one of the pre-requisites mentioned in the README file, create a **RetailData** folder within the root directory of your storage account. Upload to this folder the csv files that are placed within the folder with the same name of this repo. As you can see in the image below, your new folder in the storage account, a data lake, will be in the second level of the file system structure. 

**Did you know?**  The Synapse workspace is attached to an [ADLS Gen2 storage account](https://docs.microsoft.com/en-us/azure/storage/blobs/data-lake-storage-introduction) and the files placed on the default storage account can be accessed using the relative path as below.

<img src="https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/upload.PNG" alt="Upload" width="75%"/>

### 2. Using the Azure Portal, go to the **Access Control (IAM)** tab, click on the **+Add** and **Add a role assignment** links and add yourself to the **Contributor** role. This will allow you to create databases and tables within from your Azure Synapse Spark Pool.

### 3. Using the Azure Portal, go to [Data Explorer](https://docs.microsoft.com/en-us/azure/cosmos-db/data-explorer) of your the Azure Cosmos DB Account and create a database called **RetailSalesDemoDB**. Change the Throughput to Autoscale and set the limit to 40,000 instead of 400, this will speed-up the loading process of the data, scaling down the database when it is not in use. For more information, click [here](https://docs.microsoft.com/en-us/azure/cosmos-db/provision-throughput-autoscale).

### 4. In the same Data Explorer, create 3 **Analytical Store** enabled containers: **StoreDemoGraphics**, **RetailSales**, and **Products**. In the portal interface, the container-id is the container name. Important details:
+ Use **/id** as the Partition key for all 3 containers.
+ Please make sure that **Analytical store** is set to **On** for all 3 containers. 


<img src="https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/new-cont.PNG" alt="New Containers" width="75%"/>


### 5. In your Azure Synapse workspace, go to the **Manage / Linked Services** tab and create a service called **RetailSalesDemoDB** pointing to the database in Cosmos DB created in item 3 above.

<img src="https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/ls.PNG" alt="Surface Device" width="100%"/>

## 

### 6. Now let's load the data into Spark DataFrames.

In [ ]:
dfStoreDemoGraphics = (spark
                .read
                .csv("/RetailData/StoreDemoGraphics.csv", header=True, inferSchema='true')
              )

dfRetailSales = (spark
                .read
                .csv("/RetailData/RetailSales.csv", header=True, inferSchema='true')
              )

dfProduct = (spark
                .read
                .csv("/RetailData/Products.csv", header=True, inferSchema='true')
              )


### 7. Write the dataframe to the Azure Cosmos DB Collections

>**Did you know?** The "cosmos.oltp" is the Spark format that enables connection to the Cosmos DB Transactional store.

>**Did you know?** The ingestion to the Azure Cosmos DB collection is always performed through the Transactional store irrespective of whether the Analytical Store is enabled or not.

In [ ]:
dfStoreDemoGraphics.write\
            .format("cosmos.oltp")\
            .option("spark.synapse.linkedService", "RetailSalesDemoDB")\
            .option("spark.cosmos.container", "StoreDemoGraphics")\
            .option("spark.cosmos.write.upsertEnabled", "true")\
            .mode('append')\
            .save()

dfRetailSales.write\
            .format("cosmos.oltp")\
            .option("spark.synapse.linkedService", "RetailSalesDemoDB")\
            .option("spark.cosmos.container", "RetailSales")\
            .option("spark.cosmos.write.upsertEnabled", "true")\
            .mode('append')\
            .save()

dfProduct.write\
            .format("cosmos.oltp")\
            .option("spark.synapse.linkedService", "RetailSalesDemoDB")\
            .option("spark.cosmos.container", "Products")\
            .option("spark.cosmos.write.upsertEnabled", "true")\
            .mode('append')\
            .save()     


### 8. Using the Azure Cosmos DB Account portal, go to the Data Explorer and check if the data was loaded.

## All Done! Now let's go to the [Forecast Notebook](2SalesForecastingWithAML.ipynb).